In [ ]:
# To connect to the Jupiter Server
# In the docker attach screen, look for a message like this:
#    http://127.0.0.1:8888/lab?token=xxxx
# The "xxxx" is the password to be used when the Jupyter Kernel Connection ask for it...
# Then select the "Existing Jupiter Server" option
# Specify the URL: http://127.0.0.1:8888
# Specify the password: xxxx
# Select the desired Kernel from the list

# References:
# google: what is the default python jupiterlab server password
# https://stackoverflow.com/questions/41117554/what-is-default-password-for-jupyter-created-on-googles-data-proc

# https://github.com/jupyter/notebook/commit/7fa5d5a1be147e9c8e14f61a2f4b3c0db1e2c00b
# For servers with token-authentication enabled, the URL in the above listing will include the token,
# so you can copy and paste that URL into your browser to login.

Step 1.1: Setup Environment

In [1]:
# Init Minio
!sh /home/PyCon2024/Project/Scripts/1.init_minio.sh "data/raygun"


Data subdir: data/raygun
Source path: /home/PyCon2024/Project/data/raygun
Platform: amd64

mc already exists

Check if the bucket already exists...
]11;?\[2024-06-12 13:42:49 UTC]     0B Bronze/
[2024-06-12 13:42:49 UTC]     0B Gold/
[2024-06-12 13:42:49 UTC]     0B Raw/
[2024-06-12 13:42:49 UTC]     0B Silver/
Bucket already exists

Listing data from '/home/PyCon2024/Project/data/raygun'

total 76K
drwxr-xr-x 6 root root 192 Jun 12 10:38 .
drwxr-xr-x 9 root root 288 Jun 12 02:20 ..
-rw-rw-rw- 1 root root 16K Jun 12 10:38 934783892384.json
-rw-rw-rw- 1 root root 18K Jun 12 10:38 934783893022.json
-rw-rw-rw- 1 root root 18K Jun 12 10:38 934783896823.json
-rw-rw-rw- 1 root root 18K Jun 12 10:38 934783899185.json

Uploading data from '/home/PyCon2024/Project/data/raygun' to 'minio/data-lakehouse/Raw'

...93022.json: 67.28 KiB / 67.28 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 723.81 KiB/s 0s

In [41]:
# Install necessary packages
# !pip install pyspark
# !pip install s3fs
# !pip install minio
# !pip install pyhive
import sys
!{sys.executable} -m pip install pyspark
!{sys.executable} -m pip install s3fs
!{sys.executable} -m pip install minio
!{sys.executable} -m pip install pyhive
!{sys.executable} -m pip install trino

In [3]:
# Install dotenv to load environment variables
!{sys.executable} -m pip install python-dotenv

In [4]:
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv('minio.env')

# Access the environment variables
minio_access_key = os.getenv('MINIO_ACCESS_KEY')
minio_secret_key = os.getenv('MINIO_SECRET_KEY')
minio_endpoint = os.getenv('MINIO_ENDPOINT', "http://minio:9000")
minio_bucket_name = os.getenv('MINIO_BUCKET_NAME', "data-lakehouse")

In [5]:
print("Minio Access Key:", minio_access_key)
print("Minio Secret Key:", minio_secret_key)
print("Minio Endpoint:", minio_endpoint)
print("Minio Bucket Name:", minio_bucket_name)

Minio Access Key: minio_ak
Minio Secret Key: minio_sk
Minio Endpoint: http://minio:9000
Minio Bucket Name: data-lakehouse


In [6]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode
from minio import Minio
import os

In [7]:
import pyspark
import pyspark.sql. functions as sqlF

from sqlalchemy.sql import text
from sqlalchemy import create_engine

import pandas as pd

In [8]:
# Initialize Spark session
# spark = SparkSession.builder \
#     .appName("RaygunErrorTraceAnalysis") \
#     .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
#     .config("spark.hadoop.fs.s3a.access.key", minio_access_key) \
#     .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key) \
#     .config("spark.hadoop.fs.s3a.path.style.access", True) \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.jars.packages", "io.delta:delta-core_2.13:2.1.0") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .enableHiveSupport() \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("RaygunErrorTraceAnalysis") \
    .config("spark.driver.host", "localhost") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.3") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("hive.metastore.uris", "thrift://metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7df032da-5e69-4c7a-a1fa-c62c7639d4ab;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 264ms :: artifacts dl 14ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hadoop#hadoop-aw

In [9]:
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
 ('spark.repl.local.jars',
  'file:///root/.ivy2/jars/io.delta_delta-spark_2.12-3.1.0.jar,file:///root/.ivy2/jars/org.apache.hadoop_hadoop-aws-3.3.3.jar,file:///root/.ivy2/jars/io.delta_delta-storage-3.1.0.jar,file:///root/.ivy2/jars/org.antlr_antlr4-runtime-4.9.3.jar,file:///root/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.1026.jar,file:///root/.ivy2/jars/org.wildfly.openssl_wildfly-openssl-1.0.7.Final.jar'),
 ('spark.hadoop.fs.s3a.aws.credentials.provider',
  'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider'),
 ('spark.hadoop.fs.s3a.imp', 'org.apache.hadoop.fs.s3a.S3AFileSystem'),
 ('spark.app.submitTime', '1718199804139'),
 ('spark.app.name', 'RaygunErrorTraceAnalysis'),
 ('spark.sql.parquet.int96RebaseModeInWrite', 'CORRECTED'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.hadoop.fs.s3a.path.style.access', 'True'),
 ('spark.app.id', 'local-1718

Step 1.2: Upload Multiple JSON Files to MinIO

In [10]:
# Local directory path containing JSON files
local_directory = "../data/raygun"

# MinIO bucket name
bucket_name = minio_bucket_name

# Path to the JSON files in MinIO
json_files_path = f"s3a://{minio_bucket_name}/Raw/raygun/"

# Minio endpoint (only domian and port)
minio_endpoint_domain_port = minio_endpoint.replace('http://', '').replace('https://', '')

print("Local directory:", local_directory)
print("Minio bucket name:", bucket_name)
print("JSON files path:", json_files_path)
print("Minio endpoint (domain and port only):", minio_endpoint_domain_port)

Local directory: ../data/raygun
Minio bucket name: data-lakehouse
JSON files path: s3a://data-lakehouse/Raw/raygun/
Minio endpoint (domain and port only): minio:9000


In [11]:
# Initialize Minio client
minio_client = Minio(
    minio_endpoint_domain_port,
    access_key=minio_access_key,
    secret_key=minio_secret_key,
    secure=False
)

In [13]:
# Create the bucket if it doesn't exist
if minio_client.bucket_exists(bucket_name):
    print(f"Bucket {bucket_name} already exists")
else:
    minio_client.make_bucket(bucket_name)
    # Upload JSON files to MinIO
    for filename in os.listdir(local_directory):
        if filename.endswith(".json"):
            file_path = os.path.join(local_directory, filename)
            minio_client.fput_object(bucket_name, filename, file_path)
            print(f"Uploaded {filename} to {bucket_name}")

Bucket data-lakehouse already exists


Step 2: Read Multiple JSON Files from MinIO

In [14]:
# Read JSON files into DataFrame
print(f"Set 'df' from: {json_files_path}")
df = spark.read.option("multiline","true").json(json_files_path)

Set 'df' from: s3a://data-lakehouse/Raw/raygun/


24/06/12 13:44:12 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/06/12 13:44:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [15]:
df.printSchema()

root
 |-- Client: struct (nullable = true)
 |    |-- ClientUrl: string (nullable = true)
 |    |-- Name: string (nullable = true)
 |    |-- Version: string (nullable = true)
 |-- Context: struct (nullable = true)
 |    |-- Email: string (nullable = true)
 |    |-- FirstName: string (nullable = true)
 |    |-- FullName: string (nullable = true)
 |    |-- Identifier: string (nullable = true)
 |    |-- IsAnonymous: string (nullable = true)
 |    |-- Uuid: string (nullable = true)
 |-- Environment: struct (nullable = true)
 |    |-- UtcOffset: long (nullable = true)
 |-- Error: struct (nullable = true)
 |    |-- ClassName: string (nullable = true)
 |    |-- Data: string (nullable = true)
 |    |-- FileName: string (nullable = true)
 |    |-- InnerError: struct (nullable = true)
 |    |    |-- ClassName: string (nullable = true)
 |    |    |-- Data: string (nullable = true)
 |    |    |-- FileName: string (nullable = true)
 |    |    |-- InnerError: string (nullable = true)
 |    |    |-- M

In [16]:
# df.show(truncate=False)
df.show()

+--------------------+--------------------+-----------+--------------------+-----------+--------------------+--------------------+--------------------+----+--------------------+--------------------+-------+
|              Client|             Context|Environment|               Error|GroupingKey|         MachineName|          OccurredOn|             Request|Tags|                User|      UserCustomData|Version|
+--------------------+--------------------+-----------+--------------------+-----------+--------------------+--------------------+--------------------+----+--------------------+--------------------+-------+
|{https://github.c...|{NULL, NULL, NULL...|        {0}|{Aws\DynamoDb\Exc...|       NULL|create-other-prod...|2024-03-10T00:58:31Z|{{, , /var/www/cr...|NULL|{NULL, NULL, NULL...|{[{0, 0, [], 0, 0...| 2.12.1|
|{https://github.c...|{NULL, NULL, NULL...|        {0}|{Aws\DynamoDb\Exc...|       NULL|create-other-prod...|2024-03-10T00:58:57Z|{{, , /var/www/cr...|NULL|{NULL, NULL, NUL

Step 3: Process JSON Data

In [17]:
# Flatten the nested structure for easier analysis
df_flattened = df.select(
    col("Error.Message").alias("ErrorMessage"),
    col("Error.ClassName").alias("ErrorClassName"),
    col("Error.FileName").alias("ErrorFileName"),
    explode("Error.StackTrace").alias("StackTrace"),
    col("MachineName"),
    col("Request.HostName").alias("RequestHostName"),
    col("Request.Url").alias("RequestUrl"),
    col("Request.HttpMethod").alias("RequestHttpMethod"),
    col("Request.IpAddress").alias("RequestIpAddress"),
    col("Request.QueryString"),
    col("Request.Headers"),
    col("Request.Data")
)

In [18]:
df_flattened.printSchema()

root
 |-- ErrorMessage: string (nullable = true)
 |-- ErrorClassName: string (nullable = true)
 |-- ErrorFileName: string (nullable = true)
 |-- StackTrace: struct (nullable = true)
 |    |-- ClassName: string (nullable = true)
 |    |-- FileName: string (nullable = true)
 |    |-- LineNumber: long (nullable = true)
 |    |-- MethodName: string (nullable = true)
 |-- MachineName: string (nullable = true)
 |-- RequestHostName: string (nullable = true)
 |-- RequestUrl: string (nullable = true)
 |-- RequestHttpMethod: string (nullable = true)
 |-- RequestIpAddress: string (nullable = true)
 |-- QueryString: struct (nullable = true)
 |    |-- lac: string (nullable = true)
 |    |-- lck: string (nullable = true)
 |    |-- methods: string (nullable = true)
 |    |-- token: string (nullable = true)
 |    |-- uuid: string (nullable = true)
 |-- Headers: struct (nullable = true)
 |    |-- Accept: string (nullable = true)
 |    |-- Accept-Encoding: string (nullable = true)
 |    |-- Accept-Langu

In [19]:
df_flattened.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------+-----------------+-------------------------------------------------------------------------------------------------------------------

Step 4: Save Data into Apache Hive

In [59]:
hive_location = "/opt/hive/data/warehouse"
dest = f"{hive_location}/raygun_error_traces"

import shutil
shutil.rmtree(dest, ignore_errors=True)


In [60]:
# Save the processed data into Hive table
df_flattened \
    .write \
    .mode("overwrite") \
    .saveAsTable("raygun_error_traces")


24/06/12 15:16:17 WARN HiveExternalCatalog: Could not persist `spark_catalog`.`default`.`raygun_error_traces` in a Hive compatible way. Persisting it into Hive metastore in Spark SQL specific format.
org.apache.spark.SparkException: [CANNOT_RECOGNIZE_HIVE_TYPE] Cannot recognize hive type string: "STRUCT<ACCEPT:STRING,ACCEPT-ENCODING:STRING,ACCEPT-LANGUAGE:STRING,CONTENT-LENGTH:STRING,CONTENT-TYPE:STRING,HOST:STRING,REFERER:STRING,SEC-FETCH-DEST:STRING,SEC-FETCH-MODE:STRING,SEC-FETCH-SITE:STRING,USER-AGENT:STRING,X-AMZN-TRACE-ID:STRING,X-FORWARDED-FOR:STRING,X-FORWARDED-PORT:STRING,X-FORWARDED-PROTO:STRING>", column: `Headers`. The specified data type for the field cannot be recognized by Spark SQL. Please check the data type of the specified field and ensure that it is a valid Spark SQL data type. Refer to the Spark SQL documentation for a list of valid data types and their format. If the data type is correct, please ensure that you are using a supported version of Spark SQL.
	at org.a

In [61]:
# Verify the data is saved correctly
spark.sql("SELECT * FROM raygun_error_traces LIMIT 10").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------+-----------------+-------------------------------------------------------------------------------------------------------------------

In [73]:
# Get the IP addresses summary
spark.sql("SELECT RequestIpAddress FROM raygun_error_traces GROUP BY RequestIpAddress").show(truncate=False)

+----------------+
|RequestIpAddress|
+----------------+
|141.158.48.165  |
|174.231.133.8   |
|47.34.163.48    |
|172.58.166.179  |
+----------------+



Step 5: Query with Trino

In [62]:
# Connect to Trino

# With trino-python-client
# https://github.com/trinodb/trino-python-client

from trino.dbapi import connect
conn = connect(
    host='trino',
    port=8081,
    user='admin',
    catalog='hive',
    schema='default',
)
# Create a cursor object using the cursor() method
cursor = conn.cursor()

In [63]:
# Connect to Trino

# With SQLAlchhemy
# https://github.com/trinodb/trino-python-client?tab=readme-ov-file#sqlalchemy

# engine = create_engine('trino://admin@trino:8081')
# conn = engine.connect()

In [74]:

# Execute a query
cursor.execute("SELECT * FROM raygun_error_traces LIMIT 10")

TrinoExternalError: TrinoExternalError(type=EXTERNAL, name=HIVE_FILE_NOT_FOUND, message="Partition location does not exist: file:/opt/hive/data/warehouse/raygun_error_traces", query_id=20240612_152818_00004_7zbnr)

In [65]:
# Fetch the data
rows = cursor.fetchall()

AssertionError: 

In [ ]:
# Display the data
for row in rows:
    print(row)